In [1]:
import os
import csv

import torch
import torchvision

from dataset import CoNSePDataset, SegmentationTransform
from detection import utils
from detection.engine import train_one_epoch, evaluate
from model import get_model
from infrence import generate_predictions, output_prediction

In [2]:
tv_path = "./data/train"
test_path = "./data/test_release"
trial_path = f"output/trial_{len(os.listdir("output")) + 1}"
os.makedirs(trial_path)
train_dataset = CoNSePDataset(tv_path, transforms=SegmentationTransform(), train=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=utils.collate_fn)
val_dataset   = CoNSePDataset(tv_path, transforms=SegmentationTransform(val=True), train=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
model, device = get_model()

c:\Users\CVML_5\anaconda3\envs\DeepCVlab3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\CVML_5\anaconda3\envs\DeepCVlab3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_V2_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
csv_path = trial_path + "/result.csv"
with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        header = ['Epoch', 'Train_Loss', 'mAP']
        header += [f'class_{i}' for i in range(1, 5)]
        writer.writerow(header)

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.9)
num_epochs = 1000
per_epoches = 50
best_map = 0.0
mAP = 0.0

for epoch in range(1, num_epochs + 1):
    train_stats = train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=100)
    train_loss = train_stats.meters['loss'].global_avg
    
    coco_eval = evaluate(model, val_loader, device=device)
    mAP = coco_eval.coco_eval['bbox'].stats[0].item()

    precision = coco_eval.coco_eval['bbox'].eval['precision']
    num_classes = precision.shape[2]
    per_class_ap = []
    for k in range(num_classes):
        precision_k = precision[:, :, k, 0, 2]
        valid_precision = precision_k[precision_k > -1]
        if valid_precision.size > 0:
            ap = valid_precision.mean().item()
        else:
            ap = float('nan')
        per_class_ap.append(ap)

    if mAP > best_map:
        best_map = mAP
        torch.save(model.state_dict(), trial_path + f"/model_{epoch}_{mAP}.pth")

    with open(csv_path, 'a', newline='') as f:
        writer = csv.writer(f)
        row = [epoch, train_loss, mAP] + per_class_ap
        writer.writerow(row)

        lr_scheduler.step()
        print(f"[Epoch {epoch}] Train Loss: {train_loss:.4f}, mAP: {mAP:.4f}")

    if(epoch % per_epoches == 0):
        torch.save(model.state_dict(), trial_path + f"/epoch_{epoch}.pth")


Epoch: [1]  [ 0/24]  eta: 0:00:38  lr: 0.010000  loss: 4.8673 (4.8673)  loss_classifier: 1.5813 (1.5813)  loss_box_reg: 0.3610 (0.3610)  loss_mask: 1.1711 (1.1711)  loss_objectness: 1.4880 (1.4880)  loss_rpn_box_reg: 0.2660 (0.2660)  time: 1.5860  data: 0.6256  max mem: 12060
Epoch: [1]  [23/24]  eta: 0:00:00  lr: 0.010000  loss: 2.2791 (2.5179)  loss_classifier: 0.6053 (0.6934)  loss_box_reg: 0.5444 (0.5287)  loss_mask: 0.5707 (0.6526)  loss_objectness: 0.3070 (0.4247)  loss_rpn_box_reg: 0.1947 (0.2185)  time: 0.9568  data: 0.5534  max mem: 13415
Epoch: [1] Total time: 0:00:23 (0.9846 s / it)
creating index...
index created!
Test:  [ 0/21]  eta: 0:00:14  model_time: 0.1994 (0.1994)  evaluator_time: 0.3112 (0.3112)  time: 0.7084  data: 0.1973  max mem: 13415
Test:  [20/21]  eta: 0:00:00  model_time: 0.1065 (0.1173)  evaluator_time: 0.0361 (0.1245)  time: 0.3341  data: 0.1001  max mem: 13415
Test: Total time: 0:00:07 (0.3520 s / it)
Averaged stats: model_time: 0.1065 (0.1173)  evaluator

In [2]:
model, device = get_model()
model.load_state_dict(torch.load("output/trial_1/epoch_950.pth"))

image_folder = "data/test_release"
image_json = "data/test_image_name_to_ids.json"

coco_results = generate_predictions(model, image_folder, image_json)
output_prediction(coco_results)

c:\Users\CVML_5\anaconda3\envs\DeepCVlab3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\CVML_5\anaconda3\envs\DeepCVlab3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_V2_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
import pandas as pd
import matplotlib.pyplot as plt

# 轉為 DataFrame
df = pd.read_csv("output/trial_1/result.csv")

# 繪製 Loss 與 mAP 的趨勢圖
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(df['Epoch'], df['Train_Loss'], marker='o', label='Train Loss', color='tab:red')
plt.title("Training Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(df['Epoch'], df['mAP'], marker='o', label='mAP@[0.5:0.95]', color='tab:blue')
plt.title("mAP per Epoch")
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


AttributeError: module 'matplotlib' has no attribute 'get_data_path'

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(1,5):
    plt.plot(df["Epoch"], df[f"class_{i}"], label=f"Class {i}")

plt.title("Per-class AP across Epochs")
plt.xlabel("Epoch")
plt.ylabel("AP")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()